In [1]:
import os
import pandas as pd
import re
import traceback

# 폴더 경로
path=input("취합 할 파일이 있는 폴더 경로를 입력하세요")

# 결과를 저장할 엑셀 파일 경로
output_file = input(".xlsx를 포함한 저장할 파일의 제목을 입력하세요")
output_path = input("출력 파일을 저장할 폴더 경로를 입력하세요")

# 폴더 내의 모든 CSV 파일 목록 가져오기
csv_files = []
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('.CSV'):
            csv_files.append(os.path.join(root, file))

# 결과를 저장할 리스트 초기화
results = []

for file in csv_files:
    try:
        df = pd.read_csv(file)
        # 파일을 성공적으로 불러왔을 때 실행할 코드
    except pd.errors.ParserError:
        print("파일 '{}'에서 파싱 오류가 발생했습니다.".format(file))
        traceback.print_exc()

# # 각 CSV 파일에 대한 결과 계산
# for file in csv_files:
#     # CSV 파일을 DataFrame으로 불러오기
#     df = pd.read_csv(file)

    # DATE_TIME 칼럼을 기준으로 오름차순 정렬
    df = df.sort_values('DATE_TIME')

    # 결과 계산
    duplicate_count = df['CELL_ID'].duplicated().sum()
    duplicate_ratio = duplicate_count / len(df) * 100
    df["CELL_COUNT"]=pd.to_numeric(df["CELL_COUNT"],errors="coerce")
    NAN_count=sum(df["CELL_COUNT"].isna())
    non_increment_count = sum(df['CELL_COUNT'].diff() != 1)
    non_increment_ratio = non_increment_count / len(df) * 100
    noreturn_count = (df.iloc[:,-1] == 'NORETURN').sum()
    noreturn_ratio = noreturn_count / len(df) * 100
    normal_count=len(df[~df['CELL_ID'].duplicated() & (df.iloc[:,-1] != 'NORETURN') & (df['CELL_COUNT'].diff()==1)&(~df['CELL_ID'].isna())&(~df.iloc[:,-1].isna())])
    normal_ratio=normal_count/len(df)*100
    blank_count=df.iloc[:,-1].isna().sum()
    blankid_count=df['CELL_ID'].isna().sum()
    abnormal_count=len(df)-normal_count
    abnormal_ratio=abnormal_count/len(df)*100
    date = re.split("[._]", file)[-2]
    measurement=re.split("[._]", file)[-3]
    unit=re.split(r"\\|\.|/", file)[-3]

    # 결과를 리스트에 저장
    result = {
        '날짜': date,
        '호기': unit,
        '계측기 종류': measurement,
        '전체 데이터 행 수': len(df),
        "정상 데이터 수량":normal_count,
        '정상 데이터 비율':f'{normal_ratio:.3f}%',
        "비정상 데이터 수량":abnormal_count,
        '비정상 데이터 비율':f'{abnormal_ratio:.3f}%',
        'ID 중복 수량': duplicate_count,
        'ID 중복 비율': f'{duplicate_ratio:.3f}%',
        'NORETURN 수량': noreturn_count,
        'NORETURN 비율': f'{noreturn_ratio:.3f}%',
        'CELL_COUNT 이상 수량': non_increment_count,
        'CELL_COUNT 이상 비율': f'{non_increment_ratio:.3f}%',
        "데이터 공란보고":blank_count,
        "ID 공란보고":blankid_count,
        "형식 이상 데이터":NAN_count,
        '파일명': file,
    }
    results.append(result)


# 결과를 DataFrame으로 변환
result_df = pd.DataFrame(results)

# 결과를 엑셀 파일에 저장
output_file_path = os.path.join(output_path, output_file)
result_df.to_excel(output_file_path, sheet_name='결과', index=False)

print("결과가", output_file_path, "파일에 저장되었습니다.")

FileNotFoundError: [Errno 2] No such file or directory: 'Z:/ESNJ  계측기 로그/결과 파일\\24.08.14 ESNJ 테스트.xlsx'